In [2]:
import sys
import importlib

# Assuming the notebook is in the src folder, adjust the relative path based on your project structure
lib_path = '../lib/energy-py-linear'

sys.path.insert(0, lib_path)

# Import the energypylinear module dynamically
epl = importlib.import_module('energypylinear')

# Maximum Demand Charge

In [43]:
electric_load_mwh = [30.0, 50.0, 10.0]
electricity_prices = [0.0, 0.0, 0.0]
gas_prices = 20

site = epl.Site(
    assets=[
        epl.CHP(
            electric_efficiency_pct=1.0,
            electric_power_max_mw=50,
            electric_power_min_mw=0,
        )
    ],
    gas_prices=gas_prices,
    electricity_prices=electricity_prices,
    electric_load_mwh=electric_load_mwh,
)

no_demand_charge_simulation = site.optimize(
    verbose=3,
    objective={
        "terms": [
            {
                "asset_type": "site",
                "variable": "import_power_mwh",
                "interval_data": "electricity_prices",
            },
            {
                "asset_type": "site",
                "variable": "export_power_mwh",
                "interval_data": "electricity_prices",
                "coefficient": -1,
            },
            {
                "asset_type": "*",
                "variable": "gas_consumption_mwh",
                "interval_data": "gas_prices",
            },
        ]
    },
)

In [44]:
print(no_demand_charge_simulation.results['chp-electric_generation_mwh'])

0    0.0
1    0.0
2    0.0
Name: chp-electric_generation_mwh, dtype: float64


In [48]:
demand_charge_simulation = site.optimize(
    verbose=3,
    objective={
        "terms": [
            {
                "asset_type": "site",
                "variable": "import_power_mwh",
                "interval_data": "electricity_prices",
            },
            {
                "asset_type": "site",
                "variable": "export_power_mwh",
                "interval_data": "electricity_prices",
                "coefficient": -1,
            },
            {
                "asset_type": "*", #QUESTION : what does the * mean here? is this a wildcard that will default to all assets with gas consumption? what if we had 2 CHPs
                "variable": "gas_consumption_mwh",
                "interval_data": "gas_prices",
            },
            {
                "function": "max_many_variables",
                "variables": {
                    "asset_type": "site",
                    "variable": "import_power_mwh",
                },
                "constant": 40, #QUESTION : can the minimum peak demand that gets charged be set to 0 MW? So that the demand charge is applied to whatever the greatest peak demand is?
                                 # COMMENT: I like the idea of setting the min for the first month of a year to 0, and then setting the min to the max of the trailing 12 months. This could be very simple and useful. 
                                 # it would just require some additional logic between each simulation, where there is one simulation per monthly set of data. 
                "coefficient": 200, # It doesn't look like a change of this value impacts the profit or behavior of the model. Why is that?
                "M": max(electric_load_mwh) * 10 #QUESTION : what is the 10 used for here? why are we using the constant array of electric_load_mwh when we want to look at the dynamic import_power_mwh and
            },                                    # ... and optimize the model such that the LP decision variables condsider the demand pricing and import_power_mwh.
        ]
    },
)

In [49]:
print(
    demand_charge_simulation.results[
        ["site-electric_load_mwh", "chp-electric_generation_mwh"]
    ]
)

accounts = epl.get_accounts(demand_charge_simulation.results)

print({
    "profit": f"{accounts.profit:5.2f}", #QUESTION: why doesn't a change in the demand charge change the profit? How can I see the final profit change due to the coefficient sest in the "max_many_variables" term
    "emissions": f"{accounts.emissions:3.2f}"}) #QUESTION: same as question in the line above. 

# RESULT with coefficient = 800
#{'profit': '-200.00', 'emissions': '9.85'}

# RESULT with coefficient = 200
#{'profit': '-200.00', 'emissions': '9.85'}

   site-electric_load_mwh  chp-electric_generation_mwh
0                    30.0                          0.0
1                    50.0                         10.0
2                    10.0                          0.0
{'profit': '-200.00', 'emissions': '9.85'}


# Minimum Export Incentive

In [52]:
electric_load_mwh = [30.0, 45, 50.0, 10.0]
electricity_prices = [0.0, 0.0, 0, 0.0]
gas_prices = 20

site = epl.Site(
    assets=[
        epl.CHP(
            electric_efficiency_pct=1.0,
            electric_power_max_mw=50,
            electric_power_min_mw=0,
        )
    ],
    gas_prices=gas_prices,
    electricity_prices=electricity_prices,
    electric_load_mwh=electric_load_mwh,
)

In [53]:
no_export_incentive_simulation = site.optimize(
    verbose=3,
    objective={
        "terms": [
            {
                "asset_type": "site",
                "variable": "import_power_mwh",
                "interval_data": "electricity_prices",
            },
            {
                "asset_type": "site",
                "variable": "export_power_mwh",
                "interval_data": "electricity_prices",
                "coefficient": -1,
            },
            {
                "asset_type": "*",
                "variable": "gas_consumption_mwh",
                "interval_data": "gas_prices",
            },
        ]
    },
)

print(no_export_incentive_simulation.results['chp-electric_generation_mwh'])

0    0.0
1    0.0
2    0.0
3    0.0
Name: chp-electric_generation_mwh, dtype: float64


In [55]:
no_export_incentive_simulation = site.optimize(
    verbose=3,
    objective={
        "terms": [
            {
                "asset_type": "site",
                "variable": "import_power_mwh",
                "interval_data": "electricity_prices",
            },
            {
                "asset_type": "site",
                "variable": "export_power_mwh",
                "interval_data": "electricity_prices",
                "coefficient": -1,
            },
            {
                "asset_type": "*",
                "variable": "gas_consumption_mwh",
                "interval_data": "gas_prices",
            },
            {
                "function": "min_two_variables",

                #THE LINEAR PROGRAM VARIABLE IS PART 1 OF THE MIN_TWO_VARIABLES FUNCTION
                "a": {
                    "asset_type": "site",
                    "variable": "export_power_mwh",
                },

                # THE USER SUPPLIED CONSTANT IS PART 2 OF THE MIN_TWO_VARIABLES FUNCTION
                # In the docs you say "To demonstrate this we can look at a site where we want to incentivize a minimum export of 10 MW or greater in each interval." Did you mean to set the b constant to 10? 

                "b": 15,
                "coefficient": -200, #QUESTION: the -200 coefficient is the $200 incentive for every MWh exported? or only for MWh exported above 15 MWh? 
                "M": max(electric_load_mwh) * 10 #QUESTION: 
            },
        ]
    },
)

print(
    no_export_incentive_simulation.results[
        [
            "site-electric_load_mwh",
            "site-import_power_mwh",
            "site-export_power_mwh",
            "chp-electric_generation_mwh",
        ]
    ]
)


# if the cost of imported electricity is 0, yet the export is incentivized at $200/MWh for 15 MWh. Does it not get paid if it generates beyond the 15 MWh? Is every MWh beyond 15 MWh not incentivized?

''' THESE ARE THE ACTUAL SIMULATION RESULTS
   site-electric_load_mwh  site-import_power_mwh  site-export_power_mwh       chp-electric_generation_mwh  
0                    30.0                    0.0                   15.0                            45.0
1                    45.0                   45.0                    0.0                             0.0
2                    50.0                   50.0                    0.0                             0.0 
3                    10.0                    0.0                   15.0                            25.0
'''

# result expected for row 1: 
'''
   site-electric_load_mwh  site-import_power_mwh  site-export_power_mwh       chp-electric_generation_mwh  
0                    30.0          here     30.0            here   15.0                            15.0
1                    45.0                   45.0            here    0.0                             0.0 #Shouldn't this again be generating/exporting 15 MWh to get the 200 incentive?
2                    50.0                   50.0            here    0.0                             0.0 #Shouldn't this again be generating/exporting 15 MWh  to get the 200 incentive? 
3                    10.0                    0.0                   15.0                            25.0
'''


   site-electric_load_mwh  site-import_power_mwh  site-export_power_mwh  \
0                    30.0                    0.0                   15.0   
1                    45.0                   45.0                    0.0   
2                    50.0                   50.0                    0.0   
3                    10.0                    0.0                   15.0   

   chp-electric_generation_mwh  
0                         45.0  
1                          0.0  
2                          0.0  
3                         25.0  


'\n   site-electric_load_mwh  site-export_power_mwh  chp-electric_generation_mwh\n0                    30.0                   15.0                         15.0\n'